## Vistual inertia scheduling

In [1]:
import andes
import os

from statistics import fmean
from andes.interop.pandapower import to_pandapower
from andes.interop.pandapower import make_GSF, build_group_table
import gurobipy as gb
import pandas as pd
import numpy as np
import logging

logger = logging.getLogger(__name__)

from visopf import vis1

## Main - IEEE 39


### load case from andes excel

In [2]:
# get andes case from excel
dir_path = os.path.abspath('..')
case_path = '/VIS_opf/ieee14_vis.xlsx'
case = dir_path + case_path
ssa = andes.load(case, no_output=True)

REGCV2: unused data {'KpId': 50, 'KiId': 100, 'KpIq': 50, 'KiIq': 100}
REGCV2: unused data {'KpId': 50, 'KiId': 100, 'KpIq': 50, 'KiIq': 100}


### load norm parameter

In [3]:
# prepare nn data for visopf
data_path = dir_path + '/VIS_opf/NN_train'

fnorm = pd.read_csv(data_path + '/fnorm.csv')
pnorm = pd.read_csv(data_path + '/pnorm.csv')
norm = {'fnorm': fnorm, 'pnorm': pnorm }
norm


{'fnorm':            M          D         Fg          Rg    fnadir
 0  19.895678  12.197409  80.138509  126.078088 -0.012637
 1   5.744600   4.344583  39.902459   43.025549  0.005493,
 'pnorm':            M          D         Fg          Rg      Mvsg      Dvsg     Ppeak
 0  19.895678  12.197409  80.138509  126.078088  0.981245  2.533602  0.067277
 1   5.744600   4.344583  39.902459   43.025549  0.587552  0.867664  0.036406}

In [4]:
# example to call norm data
norm['fnorm']['M'].iloc[0] # mean
norm['fnorm']['M'].iloc[1] # std 

5.74459957980676

### load nn parameter

In [5]:
fw1 = pd.read_csv(data_path + '/fw1.csv', header=None)
fw2 = pd.read_csv(data_path + '/fw2.csv', header=None)

fb1 = pd.read_csv(data_path + '/fb1.csv', header=None)
fb2 = pd.read_csv(data_path + '/fb2.csv', header=None)

In [6]:
pw1 = pd.read_csv(data_path + '/pw1.csv', header=None)
pw2 = pd.read_csv(data_path + '/pw2.csv', header=None)

pb1 = pd.read_csv(data_path + '/pb1.csv', header=None)
pb2 = pd.read_csv(data_path + '/pb2.csv', header=None)

In [7]:
nn = {
        'fw1': fw1,
        'fw2': fw2,       
        'fb1': fb1,
        'fb2': fb2,
        'pw1': pw1,
        'pw2': pw2,
        'pb1': pb1,
        'pb2': pb2,
    }

### test opf model

In [8]:
ss = vis1(norm=norm, nn=nn, dpe=0.05)
ss.norm

Restricted license - for non-production use only - expires 2023-10-25


{'fnorm':            M          D         Fg          Rg    fnadir
 0  19.895678  12.197409  80.138509  126.078088 -0.012637
 1   5.744600   4.344583  39.902459   43.025549  0.005493,
 'pnorm':            M          D         Fg          Rg      Mvsg      Dvsg     Ppeak
 0  19.895678  12.197409  80.138509  126.078088  0.981245  2.533602  0.067277
 1   5.744600   4.344583  39.902459   43.025549  0.587552  0.867664  0.036406}

In [9]:
ss.norm['fnorm']['M'].iloc[0]

19.895678

In [10]:
ss.nn['fw1']

,0,1,2,3
0,0.053316,-0.018950,0.103133,0.005192
1,0.090899,-0.021882,-0.052110,-0.066491
2,-0.054552,0.081501,0.133793,0.073621
3,0.013614,0.007082,0.004717,-0.008138
4,0.017724,-0.007474,0.009466,-0.007022
...,...,...,...,...
59,0.009615,-0.005689,-0.002588,-0.018844
60,0.064262,-0.043027,0.112759,-0.003101
61,-0.140001,-0.125340,-0.942938,-0.133099
62,0.006305,-0.006394,-0.003145,0.004949


In [11]:
ss.nn['fw1'][1].iloc[1]
# ss.nn['fb1']

-0.021882491

In [12]:
ss.from_andes(ssa, ['PV_6', 'PV_7'])

In [13]:
ss.gen

,idx,u,name,Sn,Vn,bus,p0,pmax,pmin,v0,...,ramp30,type,p_pre,band,K,M,D,R,Mvsg,Dvsg
0,PV_2,1.0,PV_2,1.0,69.0,2,0.40000,0.5,0.1,1.03,...,600,1,0,0.4,1,8.0,0.0,0.12,0.0,0.0
1,PV_3,1.0,PV_3,1.0,69.0,3,0.40000,0.5,0.1,1.01,...,600,1,0,0.4,1,5.0,0.0,0.12,0.0,0.0
2,PV_4,1.0,PV_4,1.0,138.0,6,0.30000,1.0,0.1,1.03,...,600,1,0,0.9,1,5.0,0.0,0.05,0.0,0.0
3,PV_5,1.0,PV_5,1.0,69.0,8,0.30000,0.5,0.1,1.03,...,600,1,0,0.4,1,10.0,0.0,0.05,0.0,0.0
4,PV_6,1.0,PV_6,1.0,138.0,14,0.10000,0.1,0.0,1.01,...,600,2,0,0.1,1,0.0,0.0,0.00,0.0,3.0
5,PV_7,1.0,PV_7,1.0,138.0,12,0.10000,0.1,0.0,1.01,...,600,2,0,0.1,1,0.0,0.0,0.00,6.0,2.0
6,Slack_1,1.0,Slack_1,1.0,69.0,1,0.81442,3.0,0.5,1.03,...,600,1,0,2.5,1,8.0,0.0,0.12,0.0,0.0


In [14]:
# ss.gendict

In [15]:
ss.norm

{'fnorm':            M          D         Fg          Rg    fnadir
 0  19.895678  12.197409  80.138509  126.078088 -0.012637
 1   5.744600   4.344583  39.902459   43.025549  0.005493,
 'pnorm':            M          D         Fg          Rg      Mvsg      Dvsg     Ppeak
 0  19.895678  12.197409  80.138509  126.078088  0.981245  2.533602  0.067277
 1   5.744600   4.344583  39.902459   43.025549  0.587552  0.867664  0.036406}

In [16]:
a_test = []
for i in range(64):
    a_test.append('a'+str(i))
# a_test

In [17]:
# ss.Mvsg

In [18]:
# ss.pru

In [19]:
ss.costdict

{'PV_2': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_3': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_4': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_5': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_6': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_7': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'Slack_1': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0}}

In [20]:
ss.gendict.keys()

dict_keys(['PV_2', 'PV_3', 'PV_4', 'PV_5', 'PV_6', 'PV_7', 'Slack_1'])

In [21]:
ss.nn['fw2'][4].iloc[0]

0.009975906

In [22]:
ss.nn['fb1']

,0
0,0.288764
1,0.193274
2,0.329569
3,-0.045777
4,-0.388495
...,...
59,-0.407754
60,0.345564
61,-1.168843
62,-0.120155


In [23]:
ss.nn['fb2'][0].iloc[0]

-0.09059838

In [24]:
ss.build()

# ss.mdl.optimize()

Successfully build var.
Successfully build obj.
Successfully build cons.


In [25]:
zip(ss.zf_key, ss.af_key, ss.zf_bar)

In [26]:
ss.mdl.display()

Minimize
  <gurobi.LinExpr: 0.0>
Subject To


In [27]:
ss.get_res()

Successfully build var.
Successfully build obj.
Successfully build cons.
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 228 rows, 153 columns and 1769 nonzeros
Model fingerprint: 0x40b0b1ec
Variable types: 89 continuous, 64 integer (64 binary)
Coefficient statistics:
  Matrix range     [2e-07, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-01, 5e+00]
  RHS range        [1e-03, 1e+02]
Found heuristic solution: objective 2.2870000
Presolve removed 220 rows and 142 columns
Presolve time: 0.00s
Presolved: 8 rows, 11 columns, 39 nonzeros
Variable types: 9 continuous, 2 integer (2 binary)

Root relaxation: cutoff, 0 iterations, 0.00 seconds (0.00 work units)

Explored 1 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: 2.287 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.2

Successfully solve vis1.


,gen,pg,pru,prd
0,PV_2,0.100,0.0,0.0
1,PV_3,0.100,0.0,0.0
2,PV_4,0.100,0.0,0.0
3,PV_5,0.100,0.0,0.0
4,PV_6,0.100,0.0,0.0
5,PV_7,0.100,0.0,0.0
6,Slack_1,1.687,0.0,0.0


In [28]:
ss.load.p0.sum()

ss.rocof

ss.Msys # check whether dpe is reasonable

<gurobi.LinExpr: 5.142857142857142 + 0.14285714285714285 Mvsg[PV_6] + 0.14285714285714285 Mvsg[PV_7]>

In [29]:
ss.costdict

{'PV_2': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_3': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_4': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_5': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_6': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_7': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'Slack_1': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0}}

In [30]:
ss.zf


{'zf0': <gurobi.Var zf[zf0] (value 0.0)>,
 'zf1': <gurobi.Var zf[zf1] (value 0.2903845139326471)>,
 'zf2': <gurobi.Var zf[zf2] (value 0.0)>,
 'zf3': <gurobi.Var zf[zf3] (value 0.0)>,
 'zf4': <gurobi.Var zf[zf4] (value 0.0)>,
 'zf5': <gurobi.Var zf[zf5] (value 0.0)>,
 'zf6': <gurobi.Var zf[zf6] (value 0.7514790790118727)>,
 'zf7': <gurobi.Var zf[zf7] (value 0.5509051041506533)>,
 'zf8': <gurobi.Var zf[zf8] (value 0.0)>,
 'zf9': <gurobi.Var zf[zf9] (value 0.8105644520230719)>,
 'zf10': <gurobi.Var zf[zf10] (value 0.5575928801176224)>,
 'zf11': <gurobi.Var zf[zf11] (value 0.7277826137771208)>,
 'zf12': <gurobi.Var zf[zf12] (value 0.0)>,
 'zf13': <gurobi.Var zf[zf13] (value 0.0)>,
 'zf14': <gurobi.Var zf[zf14] (value 0.0)>,
 'zf15': <gurobi.Var zf[zf15] (value 0.5087339733179589)>,
 'zf16': <gurobi.Var zf[zf16] (value 0.48162688443593943)>,
 'zf17': <gurobi.Var zf[zf17] (value 0.03020337235283164)>,
 'zf18': <gurobi.Var zf[zf18] (value 0.0)>,
 'zf19': <gurobi.Var zf[zf19] (value 0.71112393

In [ ]:
ss.mdl.display()